In [20]:
!wget https://www.dropbox.com/scl/fi/1dm9qq6ceyz7dqrob96ix/SF_Mandarin_dataset_ver3_csv.csv?rlkey=jkngphmwkoocingpxoga0uv2v&dl=0
!pip install fuzzywuzzy
!pip install pyspellchecker

--2023-12-08 12:06:56--  https://www.dropbox.com/scl/fi/1dm9qq6ceyz7dqrob96ix/SF_Mandarin_dataset_ver3_csv.csv?rlkey=jkngphmwkoocingpxoga0uv2v
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc37c377fc653ac9a59030279a77.dl.dropboxusercontent.com/cd/0/inline/CJCT2RdGbucPzGquR4OXRLXryMJOSFd0H6miwvbWn_DX-bEnNy0ChLoLv0vi0lIZq4HkfNqbRw482aNLjz3prlb8Ntg5PDlj9JcKvLU1jdnik1JjKOPEhvZDr7oNKC1rL9vB9P7lM9wI6DBCdRR_7aJJ/file# [following]
--2023-12-08 12:06:57--  https://uc37c377fc653ac9a59030279a77.dl.dropboxusercontent.com/cd/0/inline/CJCT2RdGbucPzGquR4OXRLXryMJOSFd0H6miwvbWn_DX-bEnNy0ChLoLv0vi0lIZq4HkfNqbRw482aNLjz3prlb8Ntg5PDlj9JcKvLU1jdnik1JjKOPEhvZDr7oNKC1rL9vB9P7lM9wI6DBCdRR_7aJJ/file
Resolving uc37c377fc653ac9a59030279a77.dl.dropboxusercontent.com (uc37c377fc653ac9a59030279a77.dl.dropboxusercontent.com).

In [21]:
import pandas as pd
from fuzzywuzzy import fuzz, process
from spellchecker import SpellChecker
df = pd.read_csv('/content/SF_Mandarin_dataset_ver3_csv.csv?rlkey=jkngphmwkoocingpxoga0uv2v',sep=';')

df.dropna(inplace=True)
df['Position'] = df['Position'].apply(lambda x: x.lower())
df['Position'] = df['Position'].str.replace(',', '')
df['Position'] = df['Position'].str.rstrip()

In [22]:
df['Position'][:100]

0            начальник п
1               директор
2               директор
3               директор
4               директор
             ...        
95    старший специалист
96                 юрист
97    старший специалист
98        старший менедж
99             монтажник
Name: Position, Length: 100, dtype: object

In [3]:
# Получим все профессии, названия которых состоят из 1 слова
all_one_professions = []
for prof in df['Position'].unique():
  if len(prof.split())==1:
    all_one_professions.append(prof)
sorted(all_one_professions)

all_correct_professions = []

In [4]:
# Получим все корректные названия профессий, которые проходят проверку на орфографию с помощью spellchecker

spell = SpellChecker(language='RU')

def find_correct_professions(all_professions):
    spell = SpellChecker(language='RU')
    without_error = []

    for word in all_professions:
        if spell.correction(word) == word and (set(' .-') & set(word)) == set():
            without_error.append(word)

    return without_error
all_correct_professions = find_correct_professions(df['Position'].unique())

In [5]:
# Удалим вложения в корректных процессиях (например ['бух', 'бухгалтер'] -> ['бух'])
# Более простое решение: через множества
buf = []
for i in range(len(all_correct_professions)):
  yes_or_no = False
  for j in range(len(all_correct_professions)):
    if i!=j and all_correct_professions[i] == all_correct_professions[j][:len(all_correct_professions[i])]:
      yes_or_no = True
      break
  if not yes_or_no:
    buf.append(all_correct_professions[i])
all_correct_professions = buf
#************************************************************

In [6]:
# Формируем новые названия профессий
# Возвращает самое длинное название профессии из числа тех, которые начинаются с prefix
def find_longest_word_starting_with(prefix, word_list):
    matching_words = [word for word in word_list if word.startswith(prefix)]

    if not matching_words:
        return prefix

    longest_word = max(matching_words, key=len)
    return longest_word

spell = SpellChecker(language='RU')

In [10]:
# Непосредственно делаем замену
def f(item):
    words = item.replace("-", " ").replace("/", " ").split()

    correct_words = []

    for word in words:
        # Ищем наилучшее совпадение в списке профессий
        best_match, similarity = process.extractOne(word, all_correct_professions)
        longest_in_all_ones = find_longest_word_starting_with(word, all_one_professions)

        if len(word) <= 1 and words[-1]==word:
          continue
        if len(word) <=2 or \
             len(best_match) <= 3 or \
             len(word) == len(best_match):
          correct_words.append(word)
          continue

        if word in best_match:
          correct_words.append(best_match)
          continue

        # Если близость выше некоторого порога (может потребоваться настройка), то
        # => считаем это совпадением и заменяем слово
        if spell.correction(word) != word and similarity > 80:
            correct_words.append(best_match)
        else:
            correct_words.append(longest_in_all_ones)

    result = " ".join(correct_words)

    return result
res = df['Position'].apply(f)

In [23]:
# Сравнение старых наименований и новых
for i in range(100):
  print(f"{i}) {df['Position'].iloc[i]} | {res.iloc[i]}")

0) начальник п | начальник
1) директор | директор
2) директор | директор
3) директор | директор
4) директор | директор
5) строитель | строитель
6) директор | директор
7) менеджер | менеджер
8) менеджер | менеджер
9) самозанятая | самозанятая
10) директор | директор
11) мастер ма | мастер ма
12) директор | директор
13) самозанятая | самозанятая
14) директор | директор
15) самозанятая | самозанятая
16) начальник п | начальник
17) продавец-консультант | продавец консультант
18) менеджер | менеджер
19) управляющая | управляющая
20) старший специалист | старший специалист
21) директор | директор
22) управляющий | управляющий
23) управляющий | управляющий
24) самозанятая | самозанятая
25) менеджер | менеджер
26) продавец | продавец
27) продавец-консультант | продавец консультант
28) управляющий | управляющий
29) старший специалист | старший специалист
30) менеджер | менеджер
31) старший специалист | старший специалист
32) старший специалист | старший специалист
33) управляющий | управляющий


In [24]:
df['Position'] = res